# HW06: Transformers and Doc Embeddings

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [1]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
    return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

--2023-04-03 12:01:35--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: 'train.csv'

train.csv           100%[===================>]  28.10M  4.92MB/s    in 6.3s    

2023-04-03 12:01:45 (4.48 MB/s) - 'train.csv' saved [29470338/29470338]



,label,title,lead,text
52083,sport,"Davenport Powers Past Molik, Dementieva Falls","FILDERSTADT, Germany (Reuters) - Lindsay Dave...","Davenport Powers Past Molik, Dementieva Falls ..."
34647,sci/tech,Apple iMac G5,"1.8-GHz PowerPC G5 processor, 512MB DDR SDRAM,...","Apple iMac G5 1.8-GHz PowerPC G5 processor, 51..."
29757,world,A Beslan mother #39;s impossible choice,After more than 24 hours in captivity in Besla...,A Beslan mother #39;s impossible choice After ...
112012,business,US mobile giants in merger talks,US mobile operators Sprint and Nextel are in t...,US mobile giants in merger talks US mobile ope...
98506,sport,Sorenstam Ends the Year With Trophy (AP),AP - Despite 33 victories in the last four yea...,Sorenstam Ends the Year With Trophy (AP) AP - ...


## Hugginface Transformers

In [17]:
#prepare the data and split into training and test
import numpy as np

labels = np.sort(df.label.unique())

#perhaps should use floats here...
def one_hot_encoding(x, labels):
    return [int(l==x) for l in labels]

df['labels'] = df['label'].apply(lambda x: one_hot_encoding(x, labels))
df.head()

,label,title,lead,text,labels
52083,sport,"Davenport Powers Past Molik, Dementieva Falls","FILDERSTADT, Germany (Reuters) - Lindsay Dave...","Davenport Powers Past Molik, Dementieva Falls ...","[0, 0, 1, 0]"
34647,sci/tech,Apple iMac G5,"1.8-GHz PowerPC G5 processor, 512MB DDR SDRAM,...","Apple iMac G5 1.8-GHz PowerPC G5 processor, 51...","[0, 1, 0, 0]"
29757,world,A Beslan mother #39;s impossible choice,After more than 24 hours in captivity in Besla...,A Beslan mother #39;s impossible choice After ...,"[0, 0, 0, 1]"
112012,business,US mobile giants in merger talks,US mobile operators Sprint and Nextel are in t...,US mobile giants in merger talks US mobile ope...,"[1, 0, 0, 0]"
98506,sport,Sorenstam Ends the Year With Trophy (AP),AP - Despite 33 victories in the last four yea...,Sorenstam Ends the Year With Trophy (AP) AP - ...,"[0, 0, 1, 0]"


In [59]:
##TODO split the sample into a training and a test set 
##TODO prepare the dataset for torch.

from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split


#tokenize the text
#ev check len here, ev byta format, could use padding
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
df['tokenized'] = df['text'].apply(lambda x: tokenizer(x, return_tensors='pt', truncation=True, padding=True))

#split the data
X_train, X_test, y_train, y_test = train_test_split(df['tokenized'], df['labels'], test_size=0.3, random_state=42)

In [60]:
#doesn't feel right
print(X_train.head(), y_train)


#check for tensor
list(df['tokenized'].iloc[[1]])

#len seems to be 50 for all
len(list(df.tokenized[:30])[0].input_ids[0])

#perhaps have to convert training data
X_train = X_train.tolist()
X_test = X_test.tolist()

118159    [input_ids, attention_mask]
9006      [input_ids, attention_mask]
104343    [input_ids, attention_mask]
108450    [input_ids, attention_mask]
107612    [input_ids, attention_mask]
Name: tokenized, dtype: object 118159    [0, 0, 0, 1]
9006      [0, 0, 0, 1]
104343    [1, 0, 0, 0]
108450    [0, 0, 1, 0]
107612    [0, 1, 0, 0]
              ...     
101358    [0, 0, 1, 0]
3928      [1, 0, 0, 0]
110807    [0, 0, 0, 1]
35216     [0, 1, 0, 0]
87395     [0, 0, 0, 1]
Name: labels, Length: 7000, dtype: object


In [66]:
#be able to move tensor to device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') for mac...

cpu


In [68]:
#defining our model
from transformers import DistilBertForSequenceClassification, DistilBertConfig, AdamW
import torch

#can provide more args but should enable saving a trained model, perhaps can remove this
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

#creating a simple model, could increase dropout from 0.1 to 0.2, skipping config for now
#configuration = DistilBertConfig(output_hidden_states=True, dropout=0.2)

#initializing model, cant specify num_labels but write multilabel or switch to automodel
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)

print(model)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [85]:
#testing some dimensions on torch
test_list = [0, 0, 0, 1]
test_torch = torch.FloatTensor(test_list).reshape(1,4)
test_torch.shape

torch.Size([1, 4])

In [87]:
##TODO fit the model and print the obtained accuracy (hint: you can follow the training steps in the notebook. 
#To learn more, checkout the trainer class of huggingface transformers)

#training the model
#neat output
from tqdm import tqdm

#only cpu seemed available...
model.to(device)

nr_epochs = 1

#setting model to train
model.train()

#specify optimizer in order to backprop later - taking same as in notebook, could simplify this one
optimizer = torch.optim.Adam([
    {'params': model.distilbert.parameters(), 'lr': 1e-5},  
    {'params': model.classifier.parameters(), 'lr': 1e-3}
])

for epoch in range(nr_epochs):
    for text, label in tqdm(zip(X_train, y_train), total=len(X_train)): #for display
        text = text.to(device)
        label = torch.FloatTensor(label).reshape(1,4).to(device) #change format of labels
        pred = model(**text, labels=label) #diff len perhaps explains **
        loss = pred[0]
        loss.backward()
        optimizer.step()
        #start fresh
        optimizer.zero_grad()


100%|█████████████████████████████████████| 7000/7000 [1:32:13<00:00,  1.27it/s]


In [107]:
#trying to grasp the output
test_tensor = torch.Tensor([0,0.54,0.3,0.01]).reshape(1,4)
print(test_tensor)
type(test_tensor.argmax().tolist())
max(test_list)
test_tensor.tolist()

tensor([[0.0000, 0.5400, 0.3000, 0.0100]])


[[0.0, 0.5400000214576721, 0.30000001192092896, 0.009999999776482582]]

In [109]:
##TODO build a transformer model to do sequence classification with the goal to predict the label from the text

#lets try to see how the model does on the test data
model.eval()

#initializing predictions
predictions, labels = [], []

with torch.no_grad():
    for text, label in tqdm(zip(X_test,y_test), total=len(X_test)): #could avoid this if i didnt do multilabel
        pred = model(**text)
        labels.append(torch.Tensor(label).argmax().tolist())
        predictions.append(pred[0].argmax().tolist())
        
#taking the same as in the notebook provided
from sklearn import metrics
accuracy = metrics.accuracy_score(labels, predictions)
print ("accuracy", accuracy)
classification_report = metrics.classification_report(labels, predictions)
print (classification_report)

100%|███████████████████████████████████████| 3000/3000 [02:17<00:00, 21.88it/s]


accuracy 0.8976666666666666
              precision    recall  f1-score   support

           0       0.94      0.78      0.85       808
           1       0.81      0.94      0.87       706
           2       0.93      0.99      0.96       733
           3       0.92      0.90      0.91       753

    accuracy                           0.90      3000
   macro avg       0.90      0.90      0.90      3000
weighted avg       0.90      0.90      0.90      3000



# Doc Embedding

In [ ]:
# obtain the data
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip

!unzip sts2017.eval.v1.1.zip 
!unzip sts2017.gs.zip 

In [ ]:
# load the data

def load_STS_data():
    with open("STS2017.gs/STS.gs.track5.en-en.txt") as f:
        labels = [float(line.strip()) for line in f]
    
    text_a, text_b = [], []
    with open("STS2017.eval.v1.1/STS.input.track5.en-en.txt") as f:
        for line in f:
            line = line.strip().split("\t")
            text_a.append(line[0])
            text_b.append(line[1])
    return text_a, text_b, labels

text_a, text_b, labels = load_STS_data()
text_a[0], text_b[0], labels[0]

In [ ]:
# some utils
from scipy.stats import spearmanr
def evaluate(predictions, labels):
    print ("spearman's rank correlation", spearmanr(predictions, labels)[0])

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))


In [ ]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

In [ ]:
##TODO train Doc2Vec on the texts in the dataset
##TODO derive the word vectors for each text in the dataset
##TODO compute cosine similarity between the text pairs and evaluate spearman's rank correlation
## Don't worry if results are not satisfactory using Doc2Vec (the dataset is too small to train good embeddings)

In [ ]:
##TODO do the same with embeddings provided by spaCy

In [ ]:
##TODO do the same with SBERT embeddings